In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import _utils
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.0f}".format
pd.options.display.float_format = '{:,}'.format

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
#Cleaned & aggregated DF 
agg_parquet = data_prep.aggregated_merged_df()

/home/jovyan/data-analyses/5311/data_prep.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [3]:
agg_parquet.columns

Index(['organization_name', 'allocationamount_mean', 'allocationamount',
       'encumbered_amount', 'expendedamount', 'activebalance',
       'closedoutbalance', 'adjusted_allocationamount',
       'adjusted_expendedamount', 'adjusted_encumbered_amount',
       'adjusted_activebalance', 'adjusted_closedoutbalance', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'average_lifetime_miles_per_vehicle',
       'Automobiles', 'Bus', 'Other', 'Train', 'Van', 'automobiles_door',
       'bus_doors', 'van_doors', 'train_doors', 'doors_sum', '_0_9', '_10_12',
       '_13_15', '_16_20', '_21_25', '_26_30', '_31_60', '_16plus', '_60plus',
       'reporter_type', 'fleet_size', 'ntd_id', 'itp_id', 'caltrans_district',
       'mpo_rtpa', 'planning_authority', 'gtfs_static_status',
       'gtfs_realtime_status', 'GTFS_Status'],
      dtype='object')

In [4]:
agg_parquet.sort_values('organization_name')

,organization_name,allocationamount_mean,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,reporter_type,fleet_size,ntd_id,itp_id,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,GTFS_Status
0,Alpine County Community Development,"48,766.57142857143","341,366.0","293,366.0","159,901.31","181,464.69",0,"363,613.487575537","173,611.38404273108","315,613.44669672195","190,002.1035328059",0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,None,No Info,NaN,0,NaN,NaN,NaN,NaN,NaN,None
1,Amador Regional Transit System,"189,817.375","1,518,539.0","1,279,114.0","1,067,984.0","450,555.0",0,"1,612,119.2860307456","1,151,645.277815529","1,372,694.0821263643","460,474.0082152166",0.0,24.0,4.065359477333334,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,21.0,0.0,21.0,18.0,3.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,Rural Reporter,Medium,9R02-91000,11.0,10 - Stockton,NaN,Amador County Transportation Commission,Static OK,RT Incomplete,Only Static
2,Butte County Association of Governments,"490,548.9166666667","5,886,587.0","4,820,850.0","4,362,924.78","1,523,662.22",0,"6,267,702.554107772","4,719,990.908211823","5,201,964.646481403","1,547,711.6458959493",0.0,53.0,6.228448276,"200,297.8405",0.0,29.0,0.0,0.0,24.0,0.0,58.0,24.0,0.0,82.0,44.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Full Reporter,Medium,90208,47.0,03 - Marysville,Butte County Association of Governments,Butte County Association of Governments,Static OK,RT Incomplete,Only Static
3,Calaveras Transit Agency,"187,646.0","1,688,814.0","1,914,947.0","1,081,593.98","607,220.02",0,"1,801,548.9156123034","1,161,866.8361927115","2,058,384.9594890636","639,682.0794195919",0.0,12.0,5.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,11.0,0.0,11.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Rural Reporter,Medium,9R02-99442,0,NaN,NaN,NaN,NaN,NaN,None
4,"Central Contra Costa Transit Authority, dba: C...","50,412.0","50,412.0","50,412.0",0.0,"50,412.0",0,"54,399.71012033186",0.0,"54,399.71012033186","54,399.71012033186",0.0,206.0,3.4399548023333337,"72,814.40759999999",0.0,125.0,0.0,0.0,62.0,0.0,250.0,62.0,0.0,312.0,160.0,43.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,Full Reporter,Large,90078,0,NaN,NaN,NaN,NaN,NaN,None
5,City of Arcata,"82,111.42857142857","574,780.0","1,004,780.0","574,780.0",0.0,0,"628,747.6142894572","628,747.6142894572","1,110,688.9009036566",0.0,0.0,10.0,8.5,0.0,0.0,5.0,0.0,0.0,2.0,0.0,10.0,2.0,0.0,12.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Rural Reporter,Medium,9R02-91018,18.0,01 - Eureka,NaN,Humboldt County Association of Governments,Static OK,RT OK,Both static & RT OK
6,City of Arvin,"68,786.16666666667","412,717.0","336,683.0","263,453.0","149,264.0",0,"440,489.2274462622","287,784.84611181926","364,455.1626925159","152,704.38133444297",0.0,10.0,6.583333333000001,0.0,0.0,6.0,0.0,0.0,3.0,0.0,12.0,3.0,0.0,15.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Rural Reporter,Medium,9R02-91027,21.0,06 - Fresno,Kern Council of Governments,Kern Council of Governments,Static OK,RT Incomplete,Only Static
7,City of Auburn,"69,865.83333333333","419,195.0","358,817.0","300,665.0","118,530.0",0,"449,850.3913144346","328,588.3818194264","389,472.33989399514","121,262.00949500826",0.0,6.0,6.25,0.0,0.0,2.0,0.0,0.0,3.0,0.0,4.0,3.0,0.0,7.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Rural Reporter,Small,9R02-91032,23.0,03 - Marysville,Sacramento Area Council of Governments,Placer County Transportation Planning Agency,Static OK,RT Incomplete,Only Static
8,City of California City,"46,001.0","276,006.0","224,769.0","175,422.0","100,584.0",0,"294,482.0411233744","191,579.69484806142","243,244.99748779432","102,902.346275313

In [5]:
#Original DF 
BC_GTFS_NTD2 = data_prep.merged_dataframe()

/home/jovyan/data-analyses/5311/data_prep.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [6]:
#Grabbing only one row for each agency with original df
Aggregate1 = BC_GTFS_NTD2.groupby(['organization_name','reporter_type','GTFS', 'fleet_size', 'Is_Agency_In_BC_Only_1_means_Yes']).agg({'total_vehicles':'max',
       'average_age_of_fleet__in_years_':'max','doors_sum':'max','_0_9':'max', '_10_12':'max', '_13_15':'max', '_16_20':'max', '_21_25':'max',
       '_26_30':'max', '_31_60':'max', '_60plus':'max',}).reset_index()

## Summary Statistics Table Function

In [7]:
#more detail
def summary_stats1(df, column_wanted):
        df_summary = df.groupby([column_wanted,'organization_name','GTFS_Status']).agg({'doors_sum':'sum',
                                                                    'allocationamount':'sum', 'Automobiles':'sum', 'Bus':'sum', 'Other':'sum', 'Train':'sum', 'Van':'sum'})
        return df_summary

In [8]:
summary_stats1(agg_parquet, 'mpo_rtpa')

doors_sum  \
mpo_rtpa                                        organization_name                                GTFS_Status                      
Association of Monterey Bay Area Governments    Monterey-Salinas Transit                         Only RT                  251.0   
                                                San Benito County Local Transportation Authority Only Static               22.0   
                                                Santa Cruz Metropolitan Transit District         Only Static              218.0   
Butte County Association of Governments         Butte County Association of Governments          Only Static               82.0   
Fresno Council of Governments                   Fresno County Rural Transit Agency               None                     202.0   
Kern Council of Governments                     City of Arvin                                    Only Static               15.0   
                                                City of California City                          Only Static                8.0   
                                                City of McFarland                                Only Static                5.0   
                                                City of Ridgecrest                               Only Static               10.0   
                                                City of Shafter                                  Only Static                9.0   
                                                City of Taft                                     Only Static               10.0   
                                                City of Wasco                                    Only Static                5.0   
Kings County Association of Governments         Kings County Area Public Transit Agency          Both static & RT OK       49.0   
Merced County Association of Governments        Transit Joint Powers Authority for Merced County Only Static              101.0   
                                                Yosemite Area Regional Transportation System     Only Static               20.0   
Metropolitan Transportation Commission          City of Dixon                                    Only Static               10.0   
                                                City of Fairfield                                Both static & RT OK      108.0   
                                                City of Rio Vista                                Only Static                6.0   
                                                Marin County Transit District                    Only RT                  170.0   
                                                Napa Valley Transportation Authority             Only Static              120.0   
                                                San Mateo County Transit District                None                     908.0   
                                                Santa Clara Valley Transportation Authority      Both static & RT OK    1,457.0   
Sacramento Area Council of Governments          City of Auburn                                   Only Static                7.0   
                                                El Dorado County Transit Authority               Only Static               84.0   
                                                Yolo County Transportation District              Only Static              140.0   
                                                Yuba-Sutter Transit Authority                    Only Static               86.0   
San Diego Association of Governments            North County Transit District                    Both static & RT OK      453.0   
                                                San Diego Metropolitan Transit System            Only Static            1,748.0   
San Joaquin Council of Governments              San Joaquin Regional Transit District            Only Static              248.0   
Santa Barbara County Association of Governments City of G

In [9]:
#less detail
def summary_stats2(df, column_wanted):
        df_summary = df.groupby([column_wanted]).agg({'organization_name':'nunique','doors_sum':'sum',
                                                                    'allocationamount':'sum', 'Automobiles':'sum', 'Bus':'sum', 'Other':'sum', 'Train':'sum', 'Van':'sum'})
        df_summary = df_summary.sort_values('organization_name')
        return df_summary

In [10]:
summary_stats2(agg_parquet, 'mpo_rtpa')

,organization_name,doors_sum,allocationamount,Automobiles,Bus,Other,Train,Van
mpo_rtpa,,,,,,,,
Butte County Association of Governments,1,82.0,"5,886,587.0",0.0,29.0,0.0,0.0,24.0
Fresno Council of Governments,1,202.0,"8,776,187.0",6.0,21.0,0.0,0.0,148.0
Kings County Association of Governments,1,49.0,"1,427,870.0",0.0,18.0,0.0,0.0,13.0
San Joaquin Council of Governments,1,248.0,"2,354,211.0",0.0,103.0,0.0,0.0,42.0
Merced County Association of Governments,2,121.0,"5,021,224.0",0.0,33.0,0.0,0.0,55.0
San Diego Association of Governments,2,"2,201.0","5,292,094.0",0.0,750.0,0.0,208.0,285.0
Association of Monterey Bay Area Governments,3,491.0,"11,366,924.0",0.0,187.0,0.0,0.0,117.0
Santa Barbara County Association of Governments,3,85.0,"2,036,739.0",0.0,30.0,0.0,0.0,25.0
Sacramento Area Council of Governments,4,317.0,"6,828,716.0",0.0,130.0,0.0,0.0,57.0


In [11]:
summary_stats2(agg_parquet,'caltrans_district')

,organization_name,doors_sum,allocationamount,Automobiles,Bus,Other,Train,Van
caltrans_district,,,,,,,,
07 - Los Angeles,1,12.0,"2,706,782.0",0.0,6.0,0.0,0.0,0.0
02 - Redding,3,61.0,"5,598,366.0",0.0,20.0,0.0,0.0,21.0
09 - Bishop,3,146.0,"3,830,444.0",0.0,45.0,0.0,0.0,56.0
11 - San Diego,3,"2,260.0","7,128,856.0",0.0,766.0,0.0,208.0,312.0
10 - Stockton,4,390.0,"8,893,974.0",0.0,136.0,0.0,0.0,118.0
01 - Eureka,5,181.0,"17,992,751.0",1.0,30.0,0.0,0.0,119.0
05 - San Luis Obispo,6,576.0,"13,403,663.0",0.0,217.0,0.0,0.0,142.0
03 - Marysville,7,419.0,"14,004,918.0",1.0,159.0,0.0,0.0,99.0
04 - Oakland,7,"2,779.0","7,900,899.0",87.0,"1,032.0",0.0,99.0,343.0


In [12]:
summary_stats1(agg_parquet,'caltrans_district')

doors_sum  \
caltrans_district    organization_name                                GTFS_Status                      
01 - Eureka          City of Arcata                                   Both static & RT OK       12.0   
                     Humboldt Transit Authority                       Both static & RT OK       45.0   
                     Lake Transit Authority                           Both static & RT OK       55.0   
                     Mendocino Transit Authority                      Only Static               51.0   
                     Redwood Coast Transit Authority                  Only Static               18.0   
02 - Redding         Lassen Transit Service Agency                    Only Static               42.0   
                     Tehama County                                    Only Static               12.0   
                     Trinity County                                   Only Static                7.0   
03 - Marysville      Butte County Association of Governments          Only Static               82.0   
                     City of Auburn                                   Only Static                7.0   
                     Colusa County Transit Agency                     Only Static               12.0   
                     El Dorado County Transit Authority               Only Static               84.0   
                     Town of Truckee                                  Only Static                8.0   
                     Yolo County Transportation District              Only Static              140.0   
                     Yuba-Sutter Transit Authority                    Only Static               86.0   
04 - Oakland         City of Dixon                                    Only Static               10.0   
                     City of Fairfield                                Both static & RT OK      108.0   
                     City of Rio Vista                                Only Static                6.0   
                     Marin County Transit District                    Only RT                  170.0   
                     Napa Valley Transportation Authority             Only Static              120.0   
                     San Mateo County Transit District                None                     908.0   
                     Santa Clara Valley Transportation Authority      Both static & RT OK    1,457.0   
05 - San Luis Obispo City of Guadalupe                                Only Static               10.0   
                     City of Santa Maria                              Both static & RT OK       65.0   
                     City of Solvang                                  Only Static               10.0   
                     Monterey-Salinas Transit                         Only RT                  251.0   
                     San Benito County Local Transportation Authority Only Static               22.0   
                     Santa Cruz Metropolitan Transit District         Only Static              218.0   
06 - Fresno          City of Arvin                                    Only Static               15.0   
                     City of Dinuba                                   Only Static               17.0   
                     City of McFarland                                Only Static                5.0   
                     City of Porterville                              Only Static               63.0   
                     City of Shafter                                  Only Static                9.0   
                     City of Taft                                     Only Static               10.0   
                     City of Visalia                                  None                      86.0   
                     City of Wasco                                    Only Static                5.0   
                     City of Woodlake                                 Only Static                3.0   
                     Fresno County 

## GTFS Status Charts 

In [13]:
GTFS_orgs = agg_parquet.groupby(['GTFS_Status']).agg({'organization_name':'nunique'}).reset_index().rename(columns = {'organization_name': 'Count_of_Organizations'}).sort_values('Count_of_Organizations', ascending = False)
GTFS_orgs

,GTFS_Status,Count_of_Organizations
3,Only Static,45
1,None,27
0,Both static & RT OK,13
2,Only RT,2


In [14]:
_utils.basic_bar_chart(GTFS_orgs, 'GTFS_Status','Count_of_Organizations', 'GTFS_Status') 

alt.Chart(...)

## Number of organizations in each Caltrans district

In [15]:
Orgs_in_district = agg_parquet.groupby(['caltrans_district']).agg({'organization_name':'nunique'}).reset_index().rename(columns = {'organization_name':'count_of_organizations'})
Orgs_in_district = Orgs_in_district.sort_values(by='count_of_organizations', ascending=False)
Orgs_in_district

,caltrans_district,count_of_organizations
5,06 - Fresno,13
2,03 - Marysville,7
3,04 - Oakland,7
7,08 - San Bernardino,7
4,05 - San Luis Obispo,6
0,01 - Eureka,5
9,10 - Stockton,4
1,02 - Redding,3
8,09 - Bishop,3
10,11 - San Diego,3


In [16]:
_utils.basic_bar_chart(Orgs_in_district,'caltrans_district','count_of_organizations','caltrans_district') 

alt.Chart(...)

##  GTFS by fleet size 
* One for vehicles over 9
* One for vehicles over 15

In [17]:
def GTFS_Fleet(df, x_axis_col):
    for i in ['vehicles_older_than_9', 'vehicles_older_than_15',
       'vehicles_0_to_9', 'vehicles_percent_older_than_9',
       'vehicles_percent_older_than_15','vehicles_percent_0_to_9']:
        df_i = df[[x_axis_col, i]]
        bar_chart_i = _utils.basic_bar_chart(df_i, x_axis_col, i, x_axis_col)
        display(bar_chart_i)
    return bar_chart_i

In [18]:
vehicles_gtfs = _utils.aggregation_one(agg_parquet, 'GTFS_Status')
vehicles_gtfs

,GTFS_Status,vehicles_older_than_9,vehicles_older_than_15,vehicles_0_to_9,vehicles_percent_older_than_9,vehicles_percent_older_than_15,vehicles_percent_0_to_9
0,Both static & RT OK,801.0,441.0,"1,860.0",39.30323846908734,58.87850467289719,37.515127067365874
1,None,539.0,139.0,"1,048.0",26.44749754661433,18.55807743658211,21.137555465913675
2,Only RT,55.0,16.0,257.0,2.6987242394504416,2.1361815754339117,5.18354175070593
3,Only Static,643.0,153.0,"1,793.0",31.55053974484789,20.427236315086784,36.16377571601453


In [19]:
GTFS_Fleet(vehicles_gtfs, 'GTFS_Status')

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Fleet size by vehicle age

In [20]:
fleet = _utils.aggregation_one(agg_parquet,'fleet_size') 
fleet

,fleet_size,vehicles_older_than_9,vehicles_older_than_15,vehicles_0_to_9,vehicles_percent_older_than_9,vehicles_percent_older_than_15,vehicles_percent_0_to_9
0,Large,"1,757.0",696.0,"3,975.0",86.21197252208047,92.92389853137517,80.17345703912868
1,Medium,255.0,48.0,903.0,12.512266928361138,6.408544726301736,18.212989108511497
2,No Info,0.0,0.0,0.0,0.0,0.0,0.0
3,Small,26.0,5.0,80.0,1.2757605495583906,0.6675567423230975,1.6135538523598225


In [21]:
# don't need no info
fleet =  fleet.loc[fleet['fleet_size'] != 'No Info'] 

## Fleet Size by agencies

In [22]:
fleet_agencies = agg_parquet.groupby(['fleet_size']).agg({'organization_name':'nunique'}).reset_index()
fleet_agencies =  fleet_agencies.loc[fleet['fleet_size'] != 'No Info'] 
fleet_agencies

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

## Reporter Type of Agencies

In [ ]:
Reporter_type_agg = agg_parquet.groupby(['reporter_type']).agg({'organization_name':'nunique'}).reset_index().rename(columns = {'organization_name':'Count_of_Agencies'})
_utils.basic_bar_chart(Reporter_type_agg,'reporter_type','Count_of_Agencies', 'reporter_type') 

### Organizations that spent the most 
* For some reason other parquet does not sum stuff up correctly...

In [ ]:
Most_Money = BC_GTFS_NTD2.groupby(['organization_name','GTFS']).agg({'allocationamount':'sum'}).rename(columns = {'allocationamount': '5311_funds_received'}).reset_index().sort_values('5311_funds_received', ascending = False).head(10)
Most_Money

In [ ]:
Most_Money_Org= Most_Money['organization_name'].tolist()

In [ ]:
GTFS_Top_Ten = agg_parquet[agg_parquet.organization_name.isin(Most_Money_Org)]
GTFS_Top_Ten[['organization_name','GTFS_Status']] 

In [ ]:
_utils.basic_bar_chart(Top_ten,'organization_name','5311_funds_received', 'organization_name') 

### Funding Program

In [ ]:
bc_funds = BC_GTFS_NTD2.groupby(['funding_program']).agg({'organization_name':'nunique','allocationamount':'sum'}).rename(columns = {'organization_name':'Count_of_Organizations','allocationamount':'total_sum'}).reset_index().sort_values(by =['total_sum'])
_utils.basic_bar_chart(bc_funds,'funding_program','total_sum', 'funding_program') 